In [1]:
from hepmc import *
import numpy as np
import pandas as pd
import seaborn as sns
from itertools import product
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
eeqqg = densities.ee_qq_ng(1, 100., 5., .3)

In [3]:
rambo_mapping = phase_space.RamboOnDiet(100., 3)
mapped = phase_space.MappedDensity(eeqqg, rambo_mapping)

In [4]:
def log_target(xs):
    return -np.log(mapped.pdf(xs))

In [ ]:
# adaptive Metropolis
def adapt_schedule(t):
    if t < 5000:
        return True
    else:
        return False
    
met = AdaptiveMetropolisUpdate(5, log_target, proposals.Gaussian(5, .01), t_initial=1000, adapt_schedule=adapt_schedule)
%time met_sample = met.sample(106000, np.random.rand(5))
met_sample.data = met_sample.data[6000:] # discard warmup samples

In [ ]:
df = pd.DataFrame(met_sample.data[::100])

In [ ]:
g = sns.PairGrid(df)
#g.map(sns.kdeplot)
g.map_upper(sns.kdeplot, n_levels=60, shade=True)
g.map_lower(sns.kdeplot, n_levels=60, shade=True)
g.map_diag(sns.distplot)
plt.show()

In [10]:
nodes = 500
nlearn = 100000

xvals = np.random.random((nlearn, 5))
fvals = mapped.pdf(xvals)

# surrogate
basis = surrogate.extreme_learning.GaussianBasis(5)
log_vals = -np.ma.log(fvals)
xvals = xvals[~log_vals.mask]
log_vals = log_vals[~log_vals.mask]
# train
params = basis.extreme_learning_train(xvals, log_vals, nodes)

# surrogate
def surrogate_fn(xs):
    return basis.eval(*params, xs)[0]

# surrogate gradient
def surrogate_gradient(xs):
    return basis.eval_gradient(*params, xs)
mapped.pot_gradient = surrogate_gradient

In [7]:
xs = np.empty((1, 5))
xs[0] = np.array([.1, .2, .3, .4, .5])
surrogate_fn(xs)[0]

IndexError: invalid index to scalar variable.

In [ ]:
# adaptive Metropolis
def adapt_schedule(t):
    if t < 5000:
        return True
    else:
        return False
    
met = AdaptiveMetropolisUpdate(5, surrogate_fn, proposals.Gaussian(5, .01), t_initial=1000, adapt_schedule=adapt_schedule)
%time met_sample = met.sample(106000, np.random.rand(5))
met_sample.data = met_sample.data[6000:] # discard warmup samples

In [ ]:
df = pd.DataFrame(met_sample.data[::100])

In [ ]:
g = sns.PairGrid(df)
#g.map(sns.kdeplot)
g.map_upper(sns.kdeplot, n_levels=60, shade=True)
g.map_lower(sns.kdeplot, n_levels=60, shade=True)
g.map_diag(sns.distplot)
plt.show()

In [11]:
start = np.array([0.30527385, 0.59699739, 0.51985594, 0.57366659, 0.51096818])
sampler = hamiltonian.StaticSphericalHMC(mapped, .03, .03, 30, 30)
hmc_sample = sampler.sample(10000, start)

Event 1	(batch acceptance rate: 1.000000)
Event 2	(batch acceptance rate: 1.000000)	max sequence length: 1
Event 3	(batch acceptance rate: 1.000000)	max sequence length: 1
Event 4	(batch acceptance rate: 0.750000)	max sequence length: 2
Event 5	(batch acceptance rate: 0.600000)	max sequence length: 3
Event 6	(batch acceptance rate: 0.500000)	max sequence length: 4
Event 7	(batch acceptance rate: 0.428571)	max sequence length: 5
Event 8	(batch acceptance rate: 0.375000)	max sequence length: 6
Event 9	(batch acceptance rate: 0.333333)	max sequence length: 7
Event 10	(batch acceptance rate: 0.300000)	max sequence length: 8
Event 20	(batch acceptance rate: 0.400000)	max sequence length: 9
Event 30	(batch acceptance rate: 0.433333)	max sequence length: 9
Event 40	(batch acceptance rate: 0.500000)	max sequence length: 9
Event 50	(batch acceptance rate: 0.460000)	max sequence length: 9
Event 60	(batch acceptance rate: 0.433333)	max sequence length: 12
Event 70	(batch acceptance rate: 0.471429

/net/theorie/home/tjansse2/studium/masterarbeit/hep-monte-carlo/src/hepmc/hamiltonian/spherical_hmc.py:187: RuntimeWarning: invalid value encountered in add
  log_prob = -H_proposal + H_current


Event 200	(batch acceptance rate: 0.480000)	max sequence length: 12
Event 300	(batch acceptance rate: 0.503333)	max sequence length: 12
Event 400	(batch acceptance rate: 0.492500)	max sequence length: 12
Event 500	(batch acceptance rate: 0.478000)	max sequence length: 12
Event 600	(batch acceptance rate: 0.480000)	max sequence length: 12
Event 700	(batch acceptance rate: 0.502000)	max sequence length: 12
Event 800	(batch acceptance rate: 0.510000)	max sequence length: 12
Event 900	(batch acceptance rate: 0.518000)	max sequence length: 12
Event 1000	(batch acceptance rate: 0.518000)	max sequence length: 12
Event 2000	(batch acceptance rate: 0.444000)	max sequence length: 14
Event 3000	(batch acceptance rate: 0.534000)	max sequence length: 15
Event 4000	(batch acceptance rate: 0.466000)	max sequence length: 15
Event 5000	(batch acceptance rate: 0.478000)	max sequence length: 15
Event 6000	(batch acceptance rate: 0.540000)	max sequence length: 30
Event 7000	(batch acceptance rate: 0.52000

In [ ]:
print('estimated ESS:', util.effective_sample_size(hmc_sample, hmc_sample.mean, hmc_sample.variance))

In [12]:
mapped_hmc_sample = MarkovSample(data=rambo_mapping.map(hmc_sample.data), weights=hmc_sample.weights)
densities.export_hepmc(100., mapped_hmc_sample, "../samples/qcd/2-3/hmc.hepmc")

In [19]:
# resample
resamp_idx = np.random.choice(10000, 10000, p=hmc_sample.weights/hmc_sample.weights.sum())
hmc_resample = MarkovSample(data=hmc_sample.data[resamp_idx], weights=np.full(10000, 1./10000))

In [20]:
mapped_hmc_sample = MarkovSample(data=rambo_mapping.map(hmc_resample.data), weights=hmc_resample.weights)
densities.export_hepmc(100., mapped_hmc_sample, "../samples/qcd/2-3/hmc.hepmc")

In [14]:
print(hmc_sample.weights.min())
print(hmc_sample.weights.max())
print(hmc_sample.weights.mean())

0.02783826319716435
23967.166518026355
11.193745443787435
